# Russian Synodal Bible (1885)

## Import

### Libraries

In [16]:
import os
import re
import json
import time
import numpy as np
import pandas as pd 
from lxml import etree
import natasha
from natasha import Segmenter, MorphVocab, NewsEmbedding, NewsMorphTagger, NewsSyntaxParser, NewsNERTagger, PER, NamesExtractor, Doc

In [2]:
segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)
names_extractor = NamesExtractor(morph_vocab)

### Definitions

In [26]:
texts = './texts/fiction/'
bibleTXT = './texts/bible/bibliya-sinodalnyy-perevod.txt'
libCols = ['author','pub_year','title','text']
tokenOHCO = ['title','part_num','para_num', 'sent_num', 'token_num']
bibleOHCO = ['test', 'book', 'chap', 'verse']
bibleJSON = './texts/bible/bible.json'
bibleXML = './texts/bible/bible.xml'

## Pre-Processing

### Import Text into DF

In [4]:
with open(bibleTXT, 'r', encoding='windows-1251') as f: 
    bibleText = f.readlines()

bibliiaDf = pd.DataFrame(bibleText).rename(columns={0:'text'}).dropna()
bibliiaDf

,text
0,Библия. Синодальный перевод\n
1,Священное Писание \n
2,\n
3,\n
4,Перевод Библии на русский язык осуществлялся С...
...,...
191107,\n
191108,\n
191109,Подир — длинная одежда Иудейских первосвященни...
191110,\n


### Find Parts

In [5]:
cont_line = bibliiaDf.loc[bibliiaDf.text.str.contains('Оглавление')].index[0]
OT_line = bibliiaDf.loc[bibliiaDf.text.str.contains('Книги Ветхого Завета')].index[0]
NT_line = bibliiaDf.loc[bibliiaDf.text.str.contains('Книги Нового Завета')].index[0]
end_line = bibliiaDf.loc[bibliiaDf.text.str.contains('notes')].index[0]
vz_line = bibliiaDf.loc[bibliiaDf.text.str.contains('Ветхий Завет')].index[0]
nz_line = bibliiaDf.loc[bibliiaDf.text.str.contains('Новый Завет')].index[0]

### Tidy Up

In [6]:
bibliiaDf = bibliiaDf.loc[bibliiaDf.text != '\n']
bibliiaDf.loc[:,'text'] = bibliiaDf.loc[:,'text'].str.replace(r'\n', '')
bibliiaDf.loc[:,'text'] = bibliiaDf.loc[:,'text'].str.replace(r'\xa0', '')
bibliiaDf

/tmp/ipykernel_76198/1110360228.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  bibliiaDf.loc[:,'text'] = bibliiaDf.loc[:,'text'].str.replace(r'\n', '')
/home/xtra/.local/lib/python3.9/site-packages/pandas/core/frame.py:3602: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)
/tmp/ipykernel_76198/1110360228.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  bibliiaDf.loc[:,'text'] = bibliiaDf.loc[:,'text'].str.replace(r'\xa0', '')


,text
0,Библия. Синодальный перевод
1,Священное Писание
4,Перевод Библии на русский язык осуществлялся С...
10,БИБЛИЯ
12,Книги Священного Писания Ветхого и Нового Завета
...,...
191093,Обращенные из язычников.
191098,54
191101,Евреи из стран языческих.
191106,55


### Make Dict of Outline

{Testament: {Book: abbr}}

In [7]:
abbrevsDf = bibliiaDf.loc[vz_line+1:OT_line-1]
abbrevsDf[['abbr', 'title']] = abbrevsDf.loc[:,'text'].str.extract(r'^(\[.*\]) (.*)$', expand=True)
abbrevsDf = abbrevsDf.drop(['text'], axis=1)
bibliiaDict = {
    'Ветхий Завет': dict(zip(abbrevsDf.loc[vz_line+1:nz_line-1].title.str.strip(' '), abbrevsDf.loc[vz_line+1:nz_line-1].abbr.str.strip(r'[|]'))), 
    'Новый Завет': dict(zip(abbrevsDf.loc[nz_line+1:OT_line-1].title.str.strip(' '), abbrevsDf.loc[nz_line+1:OT_line-1].abbr.str.strip(r'[|]')))
}
del abbrevsDf

/home/xtra/.local/lib/python3.9/site-packages/pandas/core/frame.py:3602: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


### Assign Testament Label

In [8]:
bibliiaDf = bibliiaDf.loc[OT_line+1:end_line-1]
bibliiaDf['test'] = np.where(bibliiaDf.index<NT_line, 'OT', 'NT')
bibliiaDf

,text,test
422,[Быт.] Первая книга Моисея. Бытие,OT
426,Глава 1,OT
431,В начале сотворил Бог небо и землю.,OT
436,"Земля же была безвидна и пуста, и тьма над без...",OT
441,И сказал Бог: да будет свет. И стал свет.,OT
...,...,...
190641,И Дух и невеста говорят: прииди! И слышавший д...,NT
190646,И я также свидетельствую всякому слышащему сло...,NT
190651,и если кто отнимет что от слов книги пророчест...,NT
190656,"Свидетельствующий сие говорит: ей, гряду скоро...",NT


### Index All Parts

In [9]:
all_chaps = bibliiaDf.loc[bibliiaDf.text.str.contains(r'Глава \d')]
bibleBooks = [x for y in bibliiaDict.values() for x in y]
bookAbbrs = [val[abbr] for key, val in bibliiaDict.items() for abbr in val]
searchAbbrs = [r'^\['+val[abbr]+'\]' for key, val in bibliiaDict.items() for abbr in val]
abbrsList = "|".join(searchAbbrs)
books_index = bibliiaDf[bibliiaDf['text'].str.contains(abbrsList)].index
books_map = list(zip(books_index, bookAbbrs))
chaps_map = list(all_chaps.itertuples(name=None))

### Assign Book Label

In [10]:
for ind in books_map: 
    bibliiaDf.loc[ind[0], 'book'] = ind[1]

bibliiaDf['book'] = bibliiaDf.book.ffill()
bibliiaDf = bibliiaDf.drop([x[0] for x in books_map])
bibliiaDf

/home/xtra/.local/lib/python3.9/site-packages/pandas/core/frame.py:3602: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)
/home/xtra/.local/lib/python3.9/site-packages/pandas/core/indexing.py:1799: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,text,test,book
426,Глава 1,OT,Быт.
431,В начале сотворил Бог небо и землю.,OT,Быт.
436,"Земля же была безвидна и пуста, и тьма над без...",OT,Быт.
441,И сказал Бог: да будет свет. И стал свет.,OT,Быт.
446,"И увидел Бог свет, что он хорош, и отделил Бог...",OT,Быт.
...,...,...,...
190641,И Дух и невеста говорят: прииди! И слышавший д...,NT,Откр.
190646,И я также свидетельствую всякому слышащему сло...,NT,Откр.
190651,и если кто отнимет что от слов книги пророчест...,NT,Откр.
190656,"Свидетельствующий сие говорит: ей, гряду скоро...",NT,Откр.


### Assign Chapter & Verse Labels

In [11]:
for ind in chaps_map: 
    bibliiaDf.loc[ind[0], 'chap'] = ind[1]
    
bibliiaDf['chap'] = bibliiaDf.chap.ffill()
bibliiaDf = bibliiaDf.drop(x[0] for x in chaps_map)

bibliiaDf['chap'] = bibliiaDf.chap.str.extract(r'(\d{1,2})$')

bibliiaDf['verse'] = bibliiaDf.groupby(bibleOHCO[:3]).cumcount()+1

### Set Index & Remove Fluff

In [12]:
bibliiaDf = bibliiaDf.set_index(bibleOHCO)

bibliiaDf.loc[:,'text'] = bibliiaDf.loc[:,'text'].replace(r"\s\[.+\]", "", regex=True)

bibliiaDf

text
test book  chap verse                                                   
OT   Быт.  1    1                    В начале сотворил Бог небо и землю.
                2      Земля же была безвидна и пуста, и тьма над без...
                3              И сказал Бог: да будет свет. И стал свет.
                4      И увидел Бог свет, что он хорош, и отделил Бог...
                5      И назвал Бог свет днем, а тьму ночью. И был ве...
...                                                                  ...
NT   Откр. 22   17     И Дух и невеста говорят: прииди! И слышавший д...
                18     И я также свидетельствую всякому слышащему сло...
                19     и если кто отнимет что от слов книги пророчест...
                20     Свидетельствующий сие говорит: ей, гряду скоро...
                21     Благодать Господа нашего Иисуса Христа со всем...

[37343 rows x 1 columns]

### Export to JSON

In [23]:
bibliiaDf.reset_index()#.to_json(bibleJSON)

,test,book,chap,verse,text
0,OT,Быт.,1,1,В начале сотворил Бог небо и землю.
1,OT,Быт.,1,2,"Земля же была безвидна и пуста, и тьма над без..."
2,OT,Быт.,1,3,И сказал Бог: да будет свет. И стал свет.
3,OT,Быт.,1,4,"И увидел Бог свет, что он хорош, и отделил Бог..."
4,OT,Быт.,1,5,"И назвал Бог свет днем, а тьму ночью. И был ве..."
...,...,...,...,...,...
37338,NT,Откр.,22,17,И Дух и невеста говорят: прииди! И слышавший д...
37339,NT,Откр.,22,18,И я также свидетельствую всякому слышащему сло...
37340,NT,Откр.,22,19,и если кто отнимет что от слов книги пророчест...
37341,NT,Откр.,22,20,"Свидетельствующий сие говорит: ей, гряду скоро..."


In [24]:
bibleOHCO

['test', 'book', 'chap', 'verse']

In [31]:
bibliiaDf.reset_index().loc[:20].to_xml(index=True, root_name='bible', attr_cols=bibleOHCO, elem_cols=['text'])

'<?xml version=\'1.0\' encoding=\'utf-8\'?>\n<bible>\n  <row index="0" test="OT" book="Быт." chap="1" verse="1">\n    <index>0</index>\n    <text>В начале сотворил Бог небо и землю.</text>\n  </row>\n  <row index="1" test="OT" book="Быт." chap="1" verse="2">\n    <index>1</index>\n    <text>Земля же была безвидна и пуста, и тьма над бездною, и Дух Божий носился над водою.</text>\n  </row>\n  <row index="2" test="OT" book="Быт." chap="1" verse="3">\n    <index>2</index>\n    <text>И сказал Бог: да будет свет. И стал свет.</text>\n  </row>\n  <row index="3" test="OT" book="Быт." chap="1" verse="4">\n    <index>3</index>\n    <text>И увидел Бог свет, что он хорош, и отделил Бог свет от тьмы.</text>\n  </row>\n  <row index="4" test="OT" book="Быт." chap="1" verse="5">\n    <index>4</index>\n    <text>И назвал Бог свет днем, а тьму ночью. И был вечер, и было утро: день один.</text>\n  </row>\n  <row index="5" test="OT" book="Быт." chap="1" verse="6">\n    <index>5</index>\n    <text>И сказа

In [32]:
bibliiaDf.reset_index().to_xml(path_or_buffer=bibleXML, index=False, root_name='bible', attr_cols=bibleOHCO, elem_cols=['text'])

In [44]:
root = etree.Element("bible")
print(root.tag)
#bibleTree = etree.ElementTree()

bible


In [47]:
root.append(etree.Element("testament"))

In [ ]:
root.append(etree.Element(""))

In [51]:
etree.tostring(root, pretty_print=True)

b'<bible>\n  <testament/>\n</bible>\n'

In [37]:
dir(bibleTree)

['__class__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__pyx_vtable__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '_setroot',
 'docinfo',
 'find',
 'findall',
 'findtext',
 'getelementpath',
 'getiterator',
 'getpath',
 'getroot',
 'iter',
 'iterfind',
 'parse',
 'parser',
 'relaxng',
 'write',
 'write_c14n',
 'xinclude',
 'xmlschema',
 'xpath',
 'xslt']

In [39]:
bibleTree._setroot(etree.Element('bible'))

In [43]:
bibleTree.tostring()

AttributeError: 'lxml.etree._ElementTree' object has no attribute 'tostring'

## Processing

In [262]:
BibDf = pd.read_json(bibleJSON)
BibDf.index.name = 'v_id'
BibTextDf = BibDf[['text']]
BibLibDf = BibDf[bibleOHCO]
#synodalDf#.set_index(bibleOHCO)

In [263]:
BibDf

,test,book,chap,verse,text
v_id,,,,,
0,OT,Быт.,1,1,В начале сотворил Бог небо и землю.
1,OT,Быт.,1,2,"Земля же была безвидна и пуста, и тьма над без..."
2,OT,Быт.,1,3,И сказал Бог: да будет свет. И стал свет.
3,OT,Быт.,1,4,"И увидел Бог свет, что он хорош, и отделил Бог..."
4,OT,Быт.,1,5,"И назвал Бог свет днем, а тьму ночью. И был ве..."
...,...,...,...,...,...
37338,NT,Откр.,22,17,И Дух и невеста говорят: прииди! И слышавший д...
37339,NT,Откр.,22,18,И я также свидетельствую всякому слышащему сло...
37340,NT,Откр.,22,19,и если кто отнимет что от слов книги пророчест...


In [164]:
#testsDict = dict(enumerate(BibDf.test.unique()))
testsDict = dict([(value, key) for key, value in dict(enumerate(BibDf.test.unique())).items()])
booksDict = dict([(value, key) for key, value in dict(enumerate(BibDf.book.unique())).items()])

In [198]:
ChapsDf = pd.DataFrame([(x, ' '.join(y)) for (x,y) in BibDf.groupby(bibleOHCO[:3]).text], columns=[('test', 'book', 'chap'), 'text'])
ChapsDf[['test','book','chap']] = pd.DataFrame(list(ChapsDf[('test', 'book', 'chap')]), index=ChapsDf.index, columns=bibleOHCO[:3])
del ChapsDf[('test', 'book', 'chap')]
ChapsDf = ChapsDf.replace({"test": testsDict, "book": booksDict}).sort_values(by=bibleOHCO[:3], ascending=[True, True, True])
ChapsDf = ChapsDf.reset_index().drop(['index'], axis=1)
ChapsDf.index.name = 'c_id'
ChapsDf = ChapsDf.replace({"test":dict(enumerate(BibDf.test.unique())), "book":dict(enumerate(BibDf.book.unique()))})
#ChapsDf = ChapsDf.reset_index().set_index(bibleOHCO[:3])
ChapsDf

,text,test,book,chap
c_id,,,,
0,В начале сотворил Бог небо и землю. Земля же б...,OT,Быт.,1
1,Так совершены небо и земля и все воинство их. ...,OT,Быт.,2
2,"Змей был хитрее всех зверей полевых, которых с...",OT,Быт.,3
3,"Адам познал Еву, жену свою; и она зачала, и ро...",OT,Быт.,4
4,Вот родословие Адама: когда Бог сотворил челов...,OT,Быт.,5
...,...,...,...,...
1193,"После сего я увидел иного Ангела, сходящего с ...",NT,Откр.,18
1194,После сего я услышал на небе громкий голос как...,NT,Откр.,19
1195,"И увидел я Ангела, сходящего с неба, который и...",NT,Откр.,20


In [197]:
BooksDf = pd.DataFrame([(x, ' '.join(y)) for (x,y) in BibDf.groupby(bibleOHCO[:2]).text], columns=[('test', 'book'), 'text'])
BooksDf[['test','book']] = pd.DataFrame(list(BooksDf[('test', 'book')]), index=BooksDf.index, columns=bibleOHCO[:2])
del BooksDf[('test', 'book')]
BooksDf = BooksDf.replace({"test": testsDict, "book": booksDict}).sort_values(by=bibleOHCO[:2], ascending=[True, True])
BooksDf = BooksDf.reset_index().drop(['index'], axis=1)
BooksDf.index.name = 'b_id'
BooksDf = BooksDf.replace({"test":dict(enumerate(BibDf.test.unique())), "book":dict(enumerate(BibDf.book.unique()))})
#BooksDf = BooksDf.reset_index().set_index(bibleOHCO[:2])
BooksDf 

,text,test,book
b_id,,,
0,В начале сотворил Бог небо и землю. Земля же б...,OT,Быт.
1,"Вот имена сынов Израилевых, которые вошли в Ег...",OT,Исх.
2,И воззвал Господь к Моисею и сказал ему из ски...,OT,Лев.
3,"И сказал Господь Моисею в пустыне Синайской, в...",OT,Чис.
4,"Сии суть слова, которые говорил Моисей всем Из...",OT,Втор.
...,...,...,...
72,Павел и Силуан и Тимофей — Фессалоникской церк...,NT,1Тим.
73,"Павел, Апостол Иисуса Христа по повелению Бога...",NT,2Тим.
74,"Павел, волею Божиею Апостол Иисуса Христа, по ...",NT,Флм.


In [199]:
TestsDf = pd.DataFrame([(x, ' '.join(y)) for (x,y) in BibDf.groupby(bibleOHCO[:1]).text], columns=['test', 'text'])
TestsDf.index.name = 't_id'
TestsDf 

,test,text
t_id,,
0,NT,"Книги Нового Завета Родословие Иисуса Христа, ..."
1,OT,В начале сотворил Бог небо и землю. Земля же б...


In [264]:
BibTextDf

,text
v_id,
0,В начале сотворил Бог небо и землю.
1,"Земля же была безвидна и пуста, и тьма над без..."
2,И сказал Бог: да будет свет. И стал свет.
3,"И увидел Бог свет, что он хорош, и отделил Бог..."
4,"И назвал Бог свет днем, а тьму ночью. И был ве..."
...,...
37338,И Дух и невеста говорят: прииди! И слышавший д...
37339,И я также свидетельствую всякому слышащему сло...
37340,и если кто отнимет что от слов книги пророчест...


In [281]:
TestDf = BibTextDf.loc[1:1]#.to_frame()
TestDf

,text
v_id,
1,"Земля же была безвидна и пуста, и тьма над без..."


In [250]:
def nat_parse(doc_text): 
    doc = Doc(doc_text)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    for token in doc.tokens: 
        token.lemmatize(morph_vocab)
    doc.parse_syntax(syntax_parser)
    doc.tag_ner(ner_tagger)
    for sent in enumerate(doc.sents): 
        sent_num = sent[0]
        sent_text = sent[1]
        for token in enumerate([x for x in sent_text.tokens if x.pos!='PUNCT']): 
            token_num = token[0]
            token_text = token[1]
            start = token_text.start
            stop = token_text.stop
            text = token_text.text
            token_id = token_text.id
            head_id = token_text.head_id
            rel = token_text.rel
            pos = token_text.pos
            lemma = token_text.lemma
            # Animacy, Aspect, Case, Degree, Gender, Mood, Number, Person, Tense, VerbForm, Voice
            #print(token_text.feats)
            try: 
                anim = token_text.feats['Animacy']
            except: 
                anim = None
            try: 
                aspect = token_text.feats['Aspect']
            except: 
                aspect = None
            try: 
                case = token_text.feats['Case']
            except: 
                case = None
            try: 
                degree = token_text.feats['Degree']
            except: 
                degree = None
            try: 
                gender = token_text.feats['Gender']
            except: 
                gender = None
            try: 
                mood = token_text.feats['Mood']
            except: 
                mood = None
            try: 
                number = token_text.feats['Number']
            except: 
                number = None
            try: 
                person = token_text.feats['Person']
            except: 
                person = None
            try: 
                tense = token_text.feats['Tense']
            except: 
                tense = None
            try: 
                verb_form = token_text.feats['VerbForm']
            except: 
                verb_form = None
            try: 
                voice = token_text.feats['Voice']
            except: 
                voice = None
            #print(token)
        #print(sent)
    return token_num, start, stop, text, token_id, head_id, rel, pos, lemma, anim, aspect, case, degree, gender, mood, number, person, tense, verb_form, voice

In [255]:
columns = ['token_num', 'start', 'stop', 'text', 'token_id', 'head_id', 'rel', 'pos', 'lemma', 'anim', 'aspect', 'aspect', 'case', 'degree', 'gender', 'mood', 'number', 'person', 'tense', 'verb_form', 'voice']

In [282]:
temp = list(zip(*TestDf['text'].map(nat_parse)))
for i, c in enumerate(columns): 
    TestDf[c] = temp[i]

/home/xtra/.local/lib/python3.9/site-packages/pandas/core/frame.py:3602: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


IndexError: list index out of range

In [257]:
TestDf

,text,token_num,start,stop,token_id,head_id,rel,pos,lemma,anim,aspect,case,degree,gender,mood,number,person,tense,verb_form
v_id,,,,,,,,,,,,,,,,,,,
0,землю,6,29,34,1_7,1_5,conj,NOUN,земля,Inan,Acc,None,Fem,None,Sing,None,None,None,None
1,водою,15,76,81,1_18,1_16,obl,NOUN,вода,Inan,Ins,None,Masc,None,Plur,None,None,None,None
2,свет,2,36,40,2_3,2_2,nsubj,NOUN,свет,Inan,Nom,None,Masc,None,Sing,None,None,None,None
3,тьмы,12,55,59,1_15,1_11,obl,NOUN,тьма,Inan,Gen,None,Fem,None,Sing,None,None,None,None
4,один,7,69,73,2_10,2_7,parataxis,NUM,один,None,Nom,None,Masc,None,Sing,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Едема,14,68,73,1_16,1_14,nmod,PROPN,едем,Anim,Gen,None,Masc,None,Sing,None,None,None,None
96,Енох,11,112,116,2_14,2_12,parataxis,PROPN,енох,Anim,Acc,None,Masc,None,Sing,None,None,None,None
97,Ламеха,2,61,67,3_3,3_2,obj,PROPN,ламех,Anim,Acc,None,Masc,None,Sing,None,None,None,None


In [ ]:
doc = Doc(BibTextDf.loc[4].text)

In [ ]:
doc

In [ ]:
doc.segment(segmenter)

In [ ]:
doc

In [ ]:
doc.tokens

In [ ]:
doc.sents

In [ ]:
doc.sents[0].tokens

In [ ]:
doc.tag_morph(morph_tagger)

In [ ]:
doc.tokens

In [ ]:
doc.tokens[1].feats['Aspect']

In [ ]:
doc.sents[0].morph.print()

In [ ]:
[token.lemmatize(morph_vocab) for token in doc.tokens]

In [ ]:
doc.parse_syntax(syntax_parser)

In [ ]:
doc.tokens

In [ ]:
doc.sents[0].syntax.print()

In [ ]:
doc.tag_ner(ner_tagger)

In [ ]:
doc.spans

In [ ]:
doc.ner

In [ ]:
doc.ner.print()

In [ ]:
dir(doc)